In [45]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import re

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [46]:
url="https://sivers.org/book"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(
    url,
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")

In [47]:
allbooks = soup.find('section', id="allbooks")

In [48]:
books = []
for book in allbooks.find_all('div', class_='abook'):
    books.append(
        {
            'date_read': book['data-date'],
            'rating': book['data-rating'],
            'title': book['data-title']
        }
        
    )

In [49]:
df = pd.DataFrame(books)
df.head()

,date_read,rating,title
0,2017-09-27,10,Sum: Forty Tales from the Afterlives - by Davi...
1,2015-08-01,10,On Writing Well - by William Zinsser
2,2014-10-01,10,"Mindwise: How We Understand What Others Think,..."
3,2012-01-05,10,The War of Art - by Steven Pressfield
4,2011-12-08,10,"Thinking, Fast and Slow - by Daniel Kahneman"


In [50]:
# df['hyphens'] = df['title'].apply(lambda x: x.count("-"))
# df.sort_values('hyphens', ascending=False).head()

In [51]:
df['base_title'] = df['title'].apply(lambda x: ''.join(x.split(' - ')[:-1]))
df['base_title'] = df['base_title'].apply(lambda x: re.split("[:;]+", x)[0].lower().replace("-", " "))
df['base_title'] = df['base_title'].apply(lambda x: re.sub('[^A-Za-z0-9\s]+', '', x))
df['base_title'] = df['base_title'].apply(lambda text: text[text.startswith('the ') and len('the '):])
#df['base_title'] = df['base_title'].apply(lambda x: fixes[x] if x in fixes.keys() else x)

In [52]:
df['author'] = df['title'].apply(lambda x: x.split('- by ')[-1])

In [53]:
df

,date_read,rating,title,base_title,author
0,2017-09-27,10,Sum: Forty Tales from the Afterlives - by Davi...,sum,David Eagleman
1,2015-08-01,10,On Writing Well - by William Zinsser,on writing well,William Zinsser
2,2014-10-01,10,"Mindwise: How We Understand What Others Think,...",mindwise,Nicholas Epley
3,2012-01-05,10,The War of Art - by Steven Pressfield,war of art,Steven Pressfield
4,2011-12-08,10,"Thinking, Fast and Slow - by Daniel Kahneman",thinking fast and slow,Daniel Kahneman
5,2010-09-26,10,A Guide to the Good Life: The Ancient Art of S...,a guide to the good life,William Irvine
6,2009-04-03,10,The Time Paradox - by Philip Zimbardo and John...,time paradox,Philip Zimbardo and John Boyd
7,2007-07-11,10,Stumbling on Happiness - by Daniel Gilbert,stumbling on happiness,Daniel Gilbert
8,2004-02-26,10,E-Myth Revisited - by Michael Gerber,e myth revisited,Michael Gerber
9,2019-08-02,9,This Is Marketing - by Seth Godin,this is marketing,Seth Godin


In [54]:
df.to_json('derek_sivers_list.json', orient='records')